In [ ]:
import os
import numpy
from plotly import offline as plotly
from plotly import graph_objects
from scipy import stats
from statsmodels.stats import multitest

import capblood_seq
from capblood_seq import config

In [ ]:
CELL_TYPE = "Monocytes"
GENE = "LIPA"

In [ ]:
# Load the dataset. This downloads it if it doesn't exist already, and loads it into memory
dataset = capblood_seq.load_dataset(data_directory="data", pipeline_name="normalized")
dataset.filter_multi_labeled_cells(config.CELL_TYPES)

In [ ]:
traces = []

for subject_id in config.SUBJECT_IDS:
    
    x_values = []
    y_values = []
    errors = []
    
    for sample in config.SAMPLE_NAMES:
        
        gene_counts = dataset.get_transcript_counts(
            sample,
            subject_id=subject_id,
            cell_type=CELL_TYPE,
            normalized=True,
            genes=GENE
        )
        
        if gene_counts is None:
            continue
        
        gene_counts = gene_counts.to_array()
        gene_mean = gene_counts.mean()
        gene_SE = stats.sem(gene_counts)
        
        y_values.append(gene_mean)
        errors.append(gene_SE)
        x_values.append(sample)
        
    color = config.SUBJECT_ID_COLORS[subject_id]
    
    subject_scatter = graph_objects.Scatter(
        x=x_values,
        y=y_values,
        error_y={
            "array": errors,
            "thickness": 1.5,
            "color": config.SUBJECT_ID_COLORS[subject_id].replace("1.00)", "0.50)")
        },
        name=subject_id,
        line={
            "color": config.SUBJECT_ID_COLORS[subject_id]
        }
    )
    
    traces.append(subject_scatter)
    
title = "%s Expression Over Time" % GENE
if CELL_TYPE is not None:
    title += "in %s" % CELL_TYPE
    
layout = graph_objects.Layout(
    {
        "xaxis": {
            "title": "Sample"
        },
        "yaxis": {
            "title": "Mean Expression",
            "exponentformat": "power"
        },
        "plot_bgcolor": "rgba(0, 0, 0, 0)",
        "title": {
            "text": "%s Expression Over Time" % GENE
        },
        "width": 800,
        "height": 400
    }
)

figure = graph_objects.Figure(data=traces, layout=layout)
plotly.iplot(figure)

In [ ]:
figure.write_image(os.path.join("figures", "%s_expression_over_time.svg" % GENE))
figure.write_html(os.path.join("figures", "%s_expression_over_time.html" % GENE))